 # ==Important Note==
 The motion planner visualization might not work in the notebook for you. If it doesn't work, you can run the code in a python script alongside reading the notebook.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time

In [3]:
from ur_lab.motion_planning.motion_planner import MotionPlanner
from ur_lab.motion_planning.abstract_motion_planner import AbstractMotionPlanner

mp = MotionPlanner()

In [4]:
mp.visualize()
time.sleep(10)  # so other cells won't run if you press run all (which you shouldn't)

***  klampt.vis: using Qt5 as the visualization backend  ***
QtBackend: initializing app as Klamp't visualization
vis: creating GL window
######### QGLWidget setProgram ###############
#########################################
klampt.vis: Making window 0
#########################################
######### QGLWidget Initialize GL ###############


Robot names

In [5]:
mp.update_robot_config("ur5e_1", [np.pi/2, 0, -np.pi/2, 0, 0, 0])

In [6]:
mp.update_robot_config("ur5e_2", [0, -np.pi/4, 0, 0, 0, 0])

In [7]:
mp.vis_config("ur5e_2", [-np.pi/2, -np.pi/2, 0, 0, 0, 0], rgba=[1, 0, 0, 0.5])
mp.vis_config("ur5e_2", [0, 0, 0, 0, 0, 0], rgba=[0, 1, 1, 0.5], vis_name="other_config")

## feasibility check

In [8]:
collision_config = [0, 0, 0, 0, 0, 0]
mp.vis_config("ur5e_1", collision_config)
print(mp.is_config_feasible("ur5e_1", collision_config))

False


## planning motion

In [9]:
q1 = [0.03923, -1.76654, -2.27161, -2.19254, 0.40798, 3.07572]
q2 = [-0.06798, -1.84265, -2.65451, 1.33179, 1.61574, 1.64959]

mp.vis_config("ur5e_1", q1, rgba=[1, 0, 0, 0.5], vis_name="start")
mp.vis_config("ur5e_1", q2, rgba=[0, 1, 0, 0.5], vis_name="goal")

In [10]:
path = mp.plan_from_start_to_goal_config("ur5e_1", q1, q2)

planning motion...
planning took  0.9247829914093018  seconds.


In [11]:
print(path)

[[0.03923, -1.76654, -2.27161, -2.19254, 0.40798, 3.07572], [-1.4923342460083975, -1.441611218902493, -1.369542810248438, 3.413669664574308, 0.686664681239826, 1.291348697700891], [-0.06798, -1.84265, -2.65451, 1.33179, 1.61574, 1.64959]]


In [12]:
for p in path:
    mp.update_robot_config("ur5e_1", p)
    time.sleep(1)

will animate later
can't plan to or from collision...

In [13]:
path = mp.plan_from_start_to_goal_config("ur5e_1", q1, collision_config)

C:\Study\master\clair-robotics-stack\lib\site-packages\klampt\plan\robotplanning.py:252: UserWarning: Goal configuration fails ['obj collision 2 table2', 'obj collision 5 keep_out_from_ur3_zone']
  warnings.warn("Goal configuration fails {}".format(gfailures))


AttributeError: 'NoneType' object has no attribute 'space'

 if there is a direct path, it is checked and ...
 > ``NOTE: ``

## Kinematics
> ``NOTE: In world coordinates!!!``

In [14]:
R_flat, t = mp.get_forward_kinematics("ur5e_1", [0, -np.pi/2, 0, -np.pi/2, 0, 0])
R = np.array(R_flat).reshape(3, 3)

print("Rotation matrix:")
print(R)
print("Translation vector:")
print(t)

Rotation matrix:
[[-1.00000000e+00  0.00000000e+00  1.79489988e-09]
 [-1.79489988e-09  0.00000000e+00 -1.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  0.00000000e+00]]
Translation vector:
[1.7917044553320089e-10, -0.383172, 1.079229]


In [15]:
goal_position = np.array([0, -0.5, 0.5])
goal_orientation_matrix = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
goal_transform = (goal_orientation_matrix.flatten(), goal_position)
config = mp.ik_solve("ur5e_1", goal_transform)
print(config)
if config is not None:
    mp.update_robot_config("ur5e_1", config)

[1.3003001799472707, -1.5112575636413743, 1.0822593860483003, -1.1417993995448024, -1.5707961172280605, 2.871096486077435]


In [16]:
from ur_lab.motion_planning.geometry_and_transforms import GeometryAndTransforms
gt = GeometryAndTransforms(mp)

ur5e_2_position = gt.point_robot_to_world("ur5e_2", [0,0,0])
print(ur5e_2_position)

[-0.76, -1.33, 0.0]


In [17]:
above_ur5e_2_position = ur5e_2_position.copy()
above_ur5e_2_position[2] += 1
mp.show_point_vis(above_ur5e_2_position, rgba=[0, 1, 0, 0.5])
mp.show_point_vis([0,0,1], name="above origin", rgba=[1, 0, 0, 0.5])

In [18]:
%pycat ../motion_planning/motion_planner.py

In [19]:
%more ../motion_planning/klampt_world.xml

http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/


In [24]:
def destroy_mp_and_reset_vis(mp):
    '''
    Only for use here where we create multiple motion planners...
    '''

    AbstractMotionPlanner.vis_initialized = False
    from klampt import vis
    vis.clear()
    del mp

In [21]:
example_world_file = """
<?xml version="1.0" encoding="UTF-8"?>
<world>

  <robot name="robot_ceiling" file="../motion_planning/ur5e_rob/ur5e_hires.rob" position="0 0 1.99" rotateRPY="0 3.14 0"/>
  <robot name="robot_table1" file="../motion_planning/ur5e_rob/ur5e_hires.rob" position="0.7 0.7 0.01" rotateRPY="0 0 0"/>
  <robot name="robot_table2" file="../motion_planning/ur5e_rob/ur5e_hires.rob" position="-0.7 -0.7 0.01" rotateRPY="0 0 0"/>


  <rigidObject name="ceiling"  position="0 0 2.0">
    <geometry mesh="../motion_planning/objects/cube.off" scale="2 2 0.01" translate="0 0 0" />
  </rigidObject>

   <rigidObject name="table"  position="0 0 0">
    <geometry mesh="../motion_planning/objects/cube.off" scale="2 2 0.01" translate="0 0 0" />
  </rigidObject>

</world>

"""

# save to file:
with open("example_klampt_world.xml", "w") as f:
    f.write(example_world_file)

In [22]:
from ur_lab.motion_planning.abstract_motion_planner import AbstractMotionPlanner

class MyMotionPlanner(AbstractMotionPlanner):

    def _get_klampt_world_path(self):
        return "example_klampt_world.xml"

    def _add_attachments(self, robot, attachments):
        pass

In [23]:

my_mp = MyMotionPlanner()
my_mp.visualize()

In [25]:
my_mp.update_robot_config("robot_ceiling", [0, 0, 0, 0, 0, 0])

In [27]:
my_mp.robot_name_mapping.keys()

dict_keys(['robot_ceiling', 'robot_table1', 'robot_table2'])

TODO outline:

animate path as debugging tool without robots